In [2]:
from bs4 import BeautifulSoup as soup
import pandas as pd

import requests
import time
import random

def wait ():    
    time.sleep(0.5+(random.randrange(0, 10)*0.05))
    
url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": "golang",
    "page": 0
}

vacancys_data = []
vacancys_on_page = True 

while vacancys_on_page:
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    
    # ищем блоки с вакансиями и выдергиваем url
    vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
    vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)
    
    # если нашли вакансии - составляем под них структуры с данными
    if vacancys_on_page:
        for vacancy_block in vacancys_on_page:
            link_container = vacancy_block.select("a[class='bloko-link']")
            vacancys_data.append({"url": link_container[0]['href']})
    
    print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )    
    params['page'] += 1
    
    # симулируем
    wait()

page  0 , count:  15
page  1 , count:  20
page  2 , count:  20
page  3 , count:  20
page  4 , count:  20
page  5 , count:  10
page  6 , count:  20
page  7 , count:  20
page  8 , count:  20
page  9 , count:  20
page  10 , count:  20
page  11 , count:  20
page  12 , count:  20
page  13 , count:  20
page  14 , count:  20
page  15 , count:  20
page  16 , count:  20
page  17 , count:  20
page  18 , count:  20
page  19 , count:  20
page  20 , count:  20
page  21 , count:  20
page  22 , count:  20
page  23 , count:  20
page  24 , count:  20
page  25 , count:  20
page  26 , count:  20
page  27 , count:  20
page  28 , count:  20
page  29 , count:  20
page  30 , count:  20
page  31 , count:  20
page  32 , count:  20
page  33 , count:  7
page  34 , count:  0


In [10]:
# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    title = title[0] or False     
    
    if title:
        for span in title.select("span"):
            if "title" in data: 
                data["title"] = str(data["title"])+" "+str(span.get_text())
            else:
                data["title"] = str(span.get_text()) 
                        
    # зарплата бля (
    
    salary_span = response_parsed.select("span[data-qa = 'bloko-header-2'][class = 'bloko-header-2 bloko-header-2_lite']")
    salary_span = salary_span[0] or False
    
    print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1
    
    data["salary"] = str(salary_span.get_text())    
        
    # симулируем
    wait()    

        
print(data)


1  of  652  :  от 200 000 до 350 000 руб. на руки
2  of  652  :  от 25 000 до 105 000 USD до вычета налогов
3  of  652  :  от 300 000 руб. на руки
4  of  652  :  от 2 500 до 3 500 USD на руки
5  of  652  :  от 130 000 до 180 000 руб. на руки
6  of  652  :  от 300 000 до 350 000 руб. на руки
7  of  652  :  от 180 000 до 280 000 руб. до вычета налогов
8  of  652  :  от 4 000 до 6 000 EUR на руки
9  of  652  :  от 150 000 до 250 000 руб. на руки
10  of  652  :  от 150 000 до 150 000 руб. на руки
11  of  652  :  от 50 000 до 150 000 руб. на руки
12  of  652  :  до 137 931 руб. на руки
13  of  652  :  от 160 000 до 220 000 руб. на руки
14  of  652  :  от 140 000 до 200 000 руб. на руки
15  of  652  :  от 2 500 до 3 000 USD на руки
16  of  652  :  от 170 000 до 200 000 руб. на руки
17  of  652  :  от 100 000 руб. на руки
18  of  652  :  от 200 000 руб. на руки
19  of  652  :  от 80 000 до 200 000 руб. на руки
20  of  652  :  от 30 000 до 35 000 руб. до вычета налогов
21  of  652  :  до 200 0

211  of  652  :  з/п не указана
212  of  652  :  от 3 000 USD на руки
213  of  652  :  от 250 000 руб. на руки
214  of  652  :  з/п не указана
215  of  652  :  з/п не указана
216  of  652  :  з/п не указана
217  of  652  :  от 3 000 USD на руки
218  of  652  :  з/п не указана
219  of  652  :  от 180 000 руб. на руки
220  of  652  :  з/п не указана
221  of  652  :  з/п не указана
222  of  652  :  з/п не указана
223  of  652  :  з/п не указана
224  of  652  :  от 150 000 до 200 000 руб. на руки
225  of  652  :  от 50 000 до 100 000 руб. на руки
226  of  652  :  з/п не указана
227  of  652  :  от 60 000 руб. до вычета налогов
228  of  652  :  з/п не указана
229  of  652  :  з/п не указана
230  of  652  :  з/п не указана
231  of  652  :  з/п не указана
232  of  652  :  от 45 000 до 50 000 руб. на руки
233  of  652  :  з/п не указана
234  of  652  :  з/п не указана
235  of  652  :  з/п не указана
236  of  652  :  з/п не указана
237  of  652  :  з/п не указана
238  of  652  :  от 100 000 до 

444  of  652  :  з/п не указана
445  of  652  :  з/п не указана
446  of  652  :  з/п не указана
447  of  652  :  от 220 000 руб. на руки
448  of  652  :  от 140 000 до 270 000 руб. на руки
449  of  652  :  з/п не указана
450  of  652  :  з/п не указана
451  of  652  :  з/п не указана
452  of  652  :  до 150 000 руб. на руки
453  of  652  :  з/п не указана
454  of  652  :  з/п не указана
455  of  652  :  до 180 000 руб. на руки
456  of  652  :  з/п не указана
457  of  652  :  от 170 000 до 250 000 руб. на руки
458  of  652  :  з/п не указана
459  of  652  :  от 300 000 руб. до вычета налогов
460  of  652  :  з/п не указана
461  of  652  :  от 60 000 до 120 000 руб. на руки
462  of  652  :  от 150 000 до 300 000 руб. на руки
463  of  652  :  з/п не указана
464  of  652  :  от 220 000 руб. на руки
465  of  652  :  з/п не указана
466  of  652  :  з/п не указана
467  of  652  :  з/п не указана
468  of  652  :  з/п не указана
469  of  652  :  з/п не указана
470  of  652  :  з/п не указана
47

In [11]:
! git add *
!

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
